# Kakuro

## Transform integer variables into binary variables

Let us assume that we have $n$ variables $x_i$ for $i = 0, \ldots, n - 1$. To have consistent notation we start indexing from $0$. Now each of these variables has a value $0,1,2,3$. In order to express to problem using the binary variables, we define binary variables $x_{i, j}$ so that $i = 0, \ldots, n - 1$ and $j \in \left\{0,1,2,3 \right\}$. Thus we have $4n$ binary variables in the final model. The interpretation of the binary variables is simple: $x_{i,j} = 1$ if $x_i = j$. This enables us to encode integers into binary model.

## Solution proposal 1: Qubit and gate efficient approach

The first idea how to tackle the challenge is to assign a state for each binary variable $x_{i,j}$. In the end, if we measure $x_{i,j} = 1$, then we know that $x_i = j$. For example, the variable $x_{0,0}$ would correspond to the state $|0000\rangle$ in the case $n = 4$. Because the number of states grows exponentially, we would be able to represent large problems with small number of qubits. If we are able to tranform the constraint into Grover oracle which favours those states which corresponds to the solution of the problem, this would solve the problem. Anyway, constructing such oracle seems a complicated task.

I think this exercise shows an interesting and also confusing point about quantum oracles. Usually quantum oracles encode the problems that we are solving. Oracles are black-box. Because we are constructing something, which we are not supposed to know, we actually solve the problem before we even send it to the quantum computer. In that sense, the quantum computer is just a machine that we use to read the result from the oracle. I will study this property deeper later.

### Studying constraint types

Although the problem definition had some discussion online, I believe that we can divide the constaints into two classes. Because I want to be able to code the solution so that anyone without thinking anything simply inputs Kakuro constraints from the problem, I start by considering the example case from the problem definition:
$$
\begin{align}
x_0 \neq & x_1 &\quad x_0 \neq& x_2 \\
x_1 \neq & x_3 &\quad x_2 \neq& x_3 \\
x_0 + x_1 =& 3 &\quad x_0 + x_2 =& 3 \\
x_2 + x_3 =& 4 &\quad x_1 + x_3 =& 4.
\end{align}
$$

#### Constraint type 1: inequality between two variables

Obviously the first class of constraints is
$$
\begin{align}
x_0 \neq & x_1 \\
x_2 \neq& x_3 \\
x_0 \neq& x_2 \\
x_1 \neq & x_3.
\end{align}
$$

Let's focus on the first constraint $x_0 \neq x_1$. In the binary variable format, this constraint means that if $x_{i,j} = 1$ then $x_{k,j} = 0$ for $i = 0, 1$, $k = 0, 1$, $i \neq k$ and for $j = 0,1,2,3$. In other words, if we flip the phase of the state corresponding the variable $x_{0,j}$ (meaning it is part of the solution), then we do not flip variable $x_{1,j}$. If we were to flip the both variables, then it would mean $x_0 = j = x_1$ which is not allowed.

This analysis shows that we have multiple options how to flip the states corresding the variables which are in the inequality constraints. As I described in the beginning, constructing the oracle means that we solve the problem in some sense. We might not be able to read the answer from the oracle but I believe that we need to go through a solving-kind of process when constructing the oracle.

So we proceed so that we initially create all the possible correct circuits for the constraint $x_0 \neq x_1$. Then we proceed to the next constraint $x_2 \neq x_3$ and append the possible options to the circuits produced in the previous step. At each appending phase, we check if some of the previous constraints are violeted. If a constraint is violeted, we drop the circuit from the process. Finally, we are left with Grover oracles that produce the correct solution to the problem.

#### Constraint type 2: sum of variables with equality to constant 

Clearly the second class of constraints is
$$
\begin{align}
x_0 + x_1 =& 3 \\
x_2 + x_3 =& 4 \\
x_0 + x_2 =& 3 \\
x_1 + x_3 =& 4.
\end{align}
$$

Again, let's focus on the constraint $x_0 + x_1 = 3$. This means that we have two variables whose values sum up to three. We can divide this into multiple cases:

- $x_0 = 0$ and $x_1 = 3$
- $x_0 = 3$ and $x_1 = 0$
- $x_0 = 1$ and $x_1 = 2$
- $x_0 = 2$ and $x_1 = 1$

Luckily variables can hold numbers only up to three so maximum value that we can face on the right-hand side of the equality is $6$. Also, it is safe to assume that we do not sum more than three variables at the time. This reduces the number of combinations.

Let's study the case that we want to encode $x_0 = 0$ and $x_1 = 3$. This means that we want to set $x_{0,0} = 1$ and $x_{1,3} = 1$ as binary variables. If we have encoded some constraints before this constraint, flipping the states of these variables might violate some previous constraints. Now we are only left to concretely code this approach.

### Implementation

Since we are dealing with multiple circuits at the same time, it is easier to use Qiskit than Pennylane.

In [28]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library.standard_gates import MCXGate, XGate
import numpy as np

If we concentrate the actual problem, the constraints are 

$$
\begin{align}
x_0 &\neq x_1 &\quad x_0 &\neq x_2 &\quad x_1 &\neq x_3 \\
x_1 &\neq x_5 &\quad x_2 &\neq x_3 &\quad x_2 &\neq x_4 \\
x_3 &\neq x_4 &\quad x_3 &\neq x_5 &\quad x_4 &\neq x_6 \\
x_5 &\neq x_6 &&&& \\
x_0 + x_2 &= 5 &\quad x_1 + x_3 + x_5 &= 3 &\quad x_4 + x_6 &= 1 \\
x_2 + x_4 + x_3 &= 5 &\quad x_0 + x_1 &= 3 &\quad x_5 + x_6 &= 1.
\end{align}
$$
For four variables we have $4*4 = 16$ binary variables which requires $16$ states so we need exactly four query qubits in Grover's algorithm. For six variables we need five query qubits.

In [29]:
query_qubits = 4
# query_qubits = 5

inequality_constraints = [(0,1), (0,2), (1,3), (1,5), (2,3), (2,4), (3,4), (3,5), (4,6), (5,6)]
inequality_constraints_test = [(0,1), (0,2), (1,3), (2,3)]

sum_equality_constraints = [{'variables': [0, 2], 'sum': 5}, {'variables': [1, 3, 5], 'sum': 3}, {'variables': [4, 6], 'sum': 1}, {'variables': [2, 3, 4], 'sum': 5}, {'variables': [0, 1], 'sum': 3}, {'variables': [5, 6], 'sum': 1}]
sum_equality_constraints_test = [{'variables': [0, 1], 'sum': 3}, {'variables': [2, 3], 'sum': 4}, {'variables': [0, 2], 'sum': 3}, {'variables': [1, 3], 'sum': 4}]

The following function maps the binary variables $x_{i,j}$ to integers whose binary representations correspond to states. We return the binary representation as a list since we will need to access each element in the representation later.

In [30]:
def variables_to_states(i,j):
    int_repr = 4*i + j
    bin_repr = np.binary_repr(int_repr)
    return '0'*(qubits - len(bin_repr)) + bin_repr

In [40]:
# Encodes if x_3 = 2 i.e. x_3 = 2 is True if |1110> is True after measurement 
print(variables_to_states(3,2))

1110


We proceed with the following idea: Every constraint induces options how to flip phases. Whatever we inlcude in the circuit, we need to exclude something. Consider the example if $x_{0,1} = 1$ then $x_{1,1} = 0$. When we include the gate that encodes the solution $x_{0,1} = 1$, then we cannot use the gate encoding $x_{1,1} = 1$ at any part of the circuit later. This process reduces the correct oracles.

In [41]:
def compose_phase_flip_gate(i, j):
    qr = QuantumRegister(qubits + 1)
    circuit = QuantumCircuit(qr)
    ctrl = variables_to_states(i,j)
    #print(ctrl)
    mctoffoli = MCXGate(4, ctrl_state=ctrl)
    circuit.append(mctoffoli, list(range(qubits + 1)))
    return circuit

def update_bin_table(x, y, oracles):
    for oracle in oracles['oracles']:
        bin_table = oracle['bin_table']
        itemindex = np.where(bin_table[x] == 1)
        bin_table[y][itemindex] = -1

def append_to_oracles(x, y, oracles):
    oracles['seen_variables'].append(x)
    for j in range(4):
        circuit = compose_phase_flip_gate(x, j)
        bin_table = np.zeros(shape=(qubits, 4))
        bin_table[x][j] = 1
        for k in range(4):
            if k != j:
                bin_table[x][k] = -1
        bin_table[y][j] = -1
        oracles['oracles'].append({'circuit': circuit, 'bin_table' : bin_table })

def build_inequality_oracles(constraint, oracles):
    seen_variables = oracles['seen_variables']
    x = constraint[0]
    y = constraint[1]
    
    if x not in seen_variables and y not in seen_variables:
        append_to_oracles(x, y, oracles)
        append_to_oracles(y, x, oracles)
    elif x in seen_variables and y not in seen_variables:
        update_bin_table(x, y, oracles)
        append_to_oracles(y, x, oracles)  
    elif y in seen_variables and x not in seen_variables:
        update_bin_table(y, x, oracles)
        append_to_oracles(x, y, oracles)
    else:
        update_bin_table(x, y, oracles)
        update_bin_table(y, x, oracles)
    
    return oracles
    
    

def create_inequality_oracles(constraints, oracles = {'seen_variables': [], 'oracles': []}):
    for constraint in constraints:
        oracles = build_inequality_oracles(constraint, oracles)
    return oracles

In [42]:
create_inequality_oracles(inequality_constraints_test)

{'seen_variables': [0, 1, 2, 3],
 'oracles': [{'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x1e144a2eef0>,
   'bin_table': array([[ 1., -1., -1., -1.],
          [-1.,  0.,  0.,  0.],
          [-1.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]])},
  {'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x1e144a2c4f0>,
   'bin_table': array([[-1.,  1., -1., -1.],
          [ 0., -1.,  0.,  0.],
          [ 0., -1.,  0.,  0.],
          [ 0.,  0.,  0.,  0.]])},
  {'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x1e144a2df60>,
   'bin_table': array([[-1., -1.,  1., -1.],
          [ 0.,  0., -1.,  0.],
          [ 0.,  0., -1.,  0.],
          [ 0.,  0.,  0.,  0.]])},
  {'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0x1e144a2c550>,
   'bin_table': array([[-1., -1., -1.,  1.],
          [ 0.,  0.,  0., -1.],
          [ 0.,  0.,  0., -1.],
          [ 0.,  0.,  0.,  0.]])},
  {'circuit': <qiskit.circuit.quantumcircuit.QuantumCircuit at 0

## Solution proposal 2: Apply ideas from paper Grover Adaptive Search for Constrained Polynomial Binary Optimization

### Encode constraints as QUBO

### Solve QUBO with Grover search

## Solution proposal 3: Quantum machine learning – again